# MYOSAIQ assessment tools

This notebook demostrates the utilisation of the evaluation methods included in the ```myosaiq``` Python module.

Source code available on the [GitHub repository](https://www.creatis.insa-lyon.fr/Challenge/myosaiq/)


## Author

William A. Romero R. (romero@creatis.insa-lyon.fr)


## Further information

- [MYOSAIQ challenge](https://www.creatis.insa-lyon.fr/Challenge/myosaiq/)
- [FIMH 2023 website](https://fimh2023.sciencesconf.org) 
- [CREATIS](https://www.creatis.insa-lyon.fr/)

## Assess segmentation

The following code snippet illustrates the metrics calculation given a ```reference``` and ```target``` segmentations (NIfTI file format).

### Overview

1. Import ```AssessSegmentation``` from ```myosaiq``` module.
2. Create an instance of the ```AssessSegmentation``` class.
3. Calculate metrics and print results.


In [6]:
from myosaiq import AssessSegmentation

aSegmentation = AssessSegmentation("./t__data/reference/012_D8.nii.gz", # Reference segmentation (NIfTI file format)
                                   "./t__data/target/012_D8.nii.gz")    # Target segmentation (NIfTI file format)

aSegmentation.Compute()

aSegmentation.PrintSingleMetrics()


SEGMENTATION ID    LABEL       METRIC       VALUE       STD        

r_012_D8            LV         VOLUME       204.5       0          
r_012_D8            MYO        VOLUME       127.6       0          
r_012_D8            MI         VOLUME       31.6        0          
r_012_D8            MVO        VOLUME       21.3        0          

r_012_D8            LV       VOLUME AD      8.9         0          
r_012_D8            MYO      VOLUME AD      12.8        0          
r_012_D8            MI       VOLUME AD      6.1         0          
r_012_D8            MVO      VOLUME AD      6.2         0          

r_012_D8            LV          DICE        0.954       0          
r_012_D8            MYO         DICE        0.829       0          
r_012_D8            MI          DICE        0.614       0          
r_012_D8            MVO         DICE        0.763       0          

r_012_D8            LV           HD         4.688       0          
r_012_D8            MYO          HD        

## Calculate Continuous Ranked Probability Score (CRPS). 

The following code snippet illustrates the calculation of the CRPS based on a file with cumulative distributions  (CSV file format). The CRPS is computed as follows

$$
CRPS = \frac{1}{600N} \sum_{m=1}^{N} \sum_{n=0}^{599} \left( \  P(y \le n) - H(n - V_{m} \ ) \right)^{2}
$$

where  $P$ is the predicted distribution, $N$ is the number of segmentations (rows in the CSV file), $V_{m}$ the actual volume, and the Heaviside step function

$$
        H(x) = \left\{ \begin{array}{cl}
                        1 & : \ x \ge 0 \\
                        0 & : \text{Otherwise}
               \end{array} \right.
$$

Further information [Kaggle Second Annual Data Science Bowl](https://www.kaggle.com/competitions/second-annual-data-science-bowl/overview/evaluation)

### File format (CSV file)

```
    ID,      VOL,     P0,     P1,     P2,  ... , P599
709_D8, 135.4000, 0.0001, 0.0001, 0.0001,  ... , 1.000
718_D8, 189.4000, 0.0000, 0.0000, 0.0001,  ... , 1.000
727_D8, 162.4000, 0.0000, 0.0000, 0.0003,  ... , 1.000
```

### Overview

1. Import ```VolumesCDF``` from ```myosaiq``` module.
2. Create an instance of the ```VolumesCDF``` class.
3. Calculate CRPS and print results.

In [7]:
from myosaiq import VolumesCDF

volumes = VolumesCDF( "./t__data/t__LV_volumes-case_actualEst.csv" )

crps = volumes.CalcCRPS()

print("\nCRPS = %.4f\n" %  crps)

[VolumesCDF] Number of volumes: 3
[VolumesCDF] Calculating CRPS ...

	Reading data from 709_D8 (135.4000 mL) ...
	Reading data from 718_D8 (189.8000 mL) ...
	Reading data from 727_D8 (162.6000 mL) ...

CRPS = 0.0153



## Assess a set of segmentations

The following code snippet illustrates the metrics calculation given a ```reference``` and ```target``` segmentations (NIfTI file format) within a CSV file.

### Overview

1. Import ```AssessSegmentations``` from ```myosaiq``` module.
2. Create an instance of the ```AssessSegmentations``` class.
3. Calculate metrics and print results.

### Input file format (CSV file)

```
REFERENCE,TARGET
./ref/R709_D8.nii.gz,./output/T709_D8.nii.gz 
./ref/R718_D8.nii.gz,./output/T718_D8.nii.gz
./ref/R727_D8.nii.gz,./output/T727_D8.nii.gz
```


In [4]:
from myosaiq import AssessSegmentations
from IPython.display import display, HTML 

aSegmentations = AssessSegmentations( "./t__data/t__Segmentations-D8-TC00.csv" )

aSegmentations.Compute()

evaluationResults = aSegmentations.GetDataFrame()

# Retrieve specific data: Left-Ventricle volume stats
query = evaluationResults.loc[ evaluationResults['LABEL'].isin(['LV'])
                             & evaluationResults['METRIC'].isin(['VOLUME','VOLUME MAE', 'VOLUME CC'])
                             & evaluationResults['SEGMENTATION ID'].isin(['REFERENCE AVG','TARGET AVG']) ]

display( HTML(query.to_html(float_format=lambda x: '%.3f' % x)) )

[AssessSegmentations::Compute] Executing ...
[AssessSegmentations::Compute] Finished!


,SEGMENTATION ID,LABEL,METRIC,VALUE,STD
0,REFERENCE AVG,LV,VOLUME,140.500,35.700
1,REFERENCE AVG,LV,VOLUME MAE,10.800,NaN
2,REFERENCE AVG,LV,VOLUME CC,1.000,NaN
0,TARGET AVG,LV,VOLUME,149.200,34.200
1,TARGET AVG,LV,VOLUME MAE,10.800,NaN
2,TARGET AVG,LV,VOLUME CC,1.000,NaN


### Display all data

In [3]:
display( HTML(evaluationResults.to_html(float_format=lambda x: '%.3f' % x)) )

,SEGMENTATION ID,LABEL,METRIC,VALUE,STD
0,REFERENCE AVG,LV,VOLUME,140.500,35.700
1,REFERENCE AVG,LV,VOLUME MAE,10.800,NaN
2,REFERENCE AVG,LV,VOLUME CC,1.000,NaN
3,REFERENCE AVG,LV,DICE,0.910,0.035
4,REFERENCE AVG,LV,HD,7.355,1.925
5,REFERENCE AVG,LV,ASSD,0.576,0.264
6,REFERENCE AVG,MYO,VOLUME,116.000,31.300
7,REFERENCE AVG,MYO,VOLUME MAE,11.100,NaN
8,REFERENCE AVG,MYO,VOLUME CC,0.900,NaN
9,REFERENCE AVG,MYO,DICE,0.789,0.066
